# Script to read a list of images , convert into verctors and stored in Pinecone database followed by searching

--[Reference](https://blog.roboflow.com/pinecone-roboflow-inference-clip/)--

In [16]:
import tensorflow as tf
gpu_devices = tf.config.list_physical_devices('GPU')
if not gpu_devices:
    print('No GPU')
else:
    print('GPU')

No GPU


In [13]:
import glob
import uuid
from dotenv import load_dotenv
import os
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
from PIL import Image
import torch
from transformers import CLIPModel, CLIPProcessor
import cv2
import numpy as np
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array

# Load environment variables from the .env file (if present)
load_dotenv()

True

In [2]:
IMAGE_DIR = "../data/image"
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
pc = Pinecone(api_key=PINECONE_API_KEY)

In [41]:
# Load CLIP model and processor.
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

In [50]:
# Function to generate image vector.
def image_to_vector(image_path):

    image =  Image.open(image_path).convert('RGB')
    inputs = processor(images=image, return_tensors="pt")
    outputs = model(**inputs)
    print(outputs)
    image_embedding = outputs.image_embeddings.detach().cpu().numpy()[0]
    image.close()
    return image_embedding


In [17]:
def image_to_embedding(image_path):
    # Step 1: Load the pre-trained MobileNetV2 model
    model = MobileNetV2(weights='imagenet', include_top=False, pooling='avg')

    # Step 2: Read the image from the specified path
    image = cv2.imread(image_path)

    # Check if image is loaded successfully
    if image is None:
        print(f"Error: Unable to load image at {image_path}")
        return None

    # Step 3: Resize the image to 224x224 pixels (size expected by MobileNetV2)
    image_resized = cv2.resize(image, (224, 224))

    # Step 4: Convert the image to a format suitable for the model
    image_array = img_to_array(image_resized)  # Convert to array
    image_array = np.expand_dims(image_array, axis=0)  # Add batch dimension
    image_array = preprocess_input(image_array)  # Preprocess the image

    # Step 5: Generate the embedding vector using the model
    embedding_vector = model.predict(image_array)

    return embedding_vector.flatten()  # Flatten the vector to 1D

In [21]:
def getMetadata(img):
    metadata ={}
    metadata["image_name"] = img.filename.replace(f'{IMAGE_DIR}\\','')[:-4]
    metadata["image_mode"] = img.mode
    metadata["image_format"] = img.format
    metadata["image_width"], metadata["image_height"] = img.size
    metadata["image_category"] = "pizza"
    metadata["image_label"] = metadata["image_name"]
    
    return metadata  
    

In [18]:
# Convert the image to an embedding vector
embedding_vector = image_to_embedding("../data/image/margherita.jpg")

# Step 6: Print the embedding vector (or part of it)
if embedding_vector is not None:
    print("Image embedding vector representation:")
    print(embedding_vector)  # Print the entire vector
    print(f"Embedding vector shape: {embedding_vector.shape}")  # Print the shape of the vector



C:\Users\TAPAJOYTI\AppData\Local\Temp\ipykernel_144292\2742166075.py:3: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  model = MobileNetV2(weights='imagenet', include_top=False, pooling='avg')


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 9s 1us/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 132s 132s/step
Image embedding vector representation:
[0.         0.99474746 0.19308046 ... 0.671874   0.5177065  0.11683978]
Embedding vector shape: (1280,)


In [25]:
# Delete Index
pc.delete_index("test-image-index")

# Create an index called test_image
pc.create_index(
    name="test-image-index",
    dimension=1280,
    spec=ServerlessSpec(
        cloud='aws',
        region='us-east-1'
    )
)

{
    "name": "test-image-index",
    "metric": "cosine",
    "host": "test-image-index-tbbbaqp.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 1280,
    "deletion_protection": "disabled",
    "tags": null
}

In [22]:
vectors =[]

for fl in glob.glob(f"{IMAGE_DIR}/*"):
    
    vector = {}
    id = str(uuid.uuid4())
    img = Image.open(fl)
    vector["id"] = id
    vector["values"] = image_to_embedding(fl)
    vector["metadata"] = getMetadata(img)
    vectors.append(vector)
    img.close()
    
 

C:\Users\TAPAJOYTI\AppData\Local\Temp\ipykernel_144292\2742166075.py:3: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  model = MobileNetV2(weights='imagenet', include_top=False, pooling='avg')


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step


In [23]:
print(vectors)

[{'id': '3a446167-c492-4083-88c5-66d3d1ac868c', 'values': array([0.00336485, 0.        , 0.30892482, ..., 3.2588277 , 0.4672242 ,
       0.5803512 ], shape=(1280,), dtype=float32), 'metadata': {'image_name': 'classic-cheese-pizza', 'image_mode': 'RGB', 'image_format': 'JPEG', 'image_width': 1500, 'image_height': 1000, 'image_category': 'pizza', 'image_label': 'classic-cheese-pizza'}}, {'id': '4e481108-667b-49b7-b171-da8415960acd', 'values': array([0.        , 0.99474746, 0.19308046, ..., 0.671874  , 0.5177065 ,
       0.11683978], shape=(1280,), dtype=float32), 'metadata': {'image_name': 'margherita', 'image_mode': 'RGB', 'image_format': 'JPEG', 'image_width': 225, 'image_height': 225, 'image_category': 'pizza', 'image_label': 'margherita'}}, {'id': 'a2c016e8-6f69-47a3-8df2-3c6a9b63e4cc', 'values': array([0.29437202, 0.        , 0.        , ..., 2.0288527 , 0.13304813,
       1.608125  ], shape=(1280,), dtype=float32), 'metadata': {'image_name': 'neapolitan', 'image_mode': 'RGB', 'imag

In [26]:
index = pc.Index("test-image-index")
index.upsert(vectors=vectors)


{'dimension': 1280,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 0}},
 'total_vector_count': 0}


In [27]:
print(index.describe_index_stats())

{'dimension': 1280,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 3}},
 'total_vector_count': 3}
